In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import h5py
import glob
from scipy.constants import physical_constants

au2fs = physical_constants['atomic unit of time'][0] * 1e15  

# Shannon entropy function
def shannon_entropy_from_pop(pop):
    p = np.array(pop, dtype=float)
    p_safe = np.where(p > 0.0, p, 1.0)   # avoid log(0)
    S = -np.sum(p * np.log(p_safe), axis=1)
    return S

# Normalization factor: N = number of states
N = 40
norm_factor = np.log(N)

plt.figure(figsize=(8, 6))

for method in ['FSSH', 'IDA', 'MSDM', 'FSSH2', 'GFSH', 'MSDM_GFSH']:
    plt.rcParams.update({"font.size": 20, "lines.linewidth": 4.0})
    files = glob.glob(f'{method}_latestnewNBRA_icond_*')  
    c = 0
    
    for file in files:
        with h5py.File(f'{file}/mem_data.hdf') as F:
            sh_pop = np.array(F['sh_pop_adi/data'])
            time_1 = np.array(F['time/data']) * au2fs 
        
        # compute entropy
        shannon_entropy = shannon_entropy_from_pop(sh_pop)

        # Normalize by ln(N)
        shannon_entropy /= norm_factor

        # accumulate average
        if c == 0:
            ave_shannon_entropy = np.zeros(shannon_entropy.shape)
        ave_shannon_entropy += shannon_entropy
        c += 1
    
    ave_shannon_entropy /= len(files)
    plt.plot(time_1, ave_shannon_entropy, label=f'{method}')

plt.title('C$_{20}$')
leg = plt.legend(fontsize=18, ncol=3, loc='lower right', 
                 frameon=False, handlelength=2.0, handleheight=2.0)

for line in leg.get_lines():
    line.set_linewidth(6.0)

plt.ylabel('Shannon Entropy')
plt.xlabel('Time, fs')
plt.tight_layout()
plt.savefig('c20-sh.jpg', dpi=600)
plt.show()
